<a href="https://colab.research.google.com/github/leandrobarbieri/pydata-book/blob/2nd-edition/Group_Aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agrupamento e agregação
- 1: Separa os grupos, 2: Aplica funções de agregação, 3: Recombina em um novo df agregado.
- Cálculos estatísticos de resumo em colunas numéricas
- Iteração nos grupos
- Agrupamentos de colunas. Une várias colunas em uma
- Agregaçao com funções
- Agregaçao com mais de uma função por coluna numérica
- Agregação de cada coluna numerica com uma fução diferente
- Tabelas pivot e crosstab

In [114]:
# importações e configurações
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)

# Group By

In [115]:
# cria um df de testes com dados categoricos (serão agrupado) e numéricos (serão agregados)
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.arange(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0,-0.204708
1,a,two,1,0.478943
2,b,one,2,-0.519439
3,b,two,3,-0.555730
4,a,one,4,1.965781


In [116]:
# primeiramente os grupos são criados com base nas colunas passadas para groupby
# um obj do tipo pandas.core.groupby.generic.SeriesGroupBy é criado
# esse obj pode ser iterado
grouped = df.groupby(df['key1'])

for g in grouped:
  print(g)

('a',   key1 key2  data1     data2
0    a  one      0 -0.204708
1    a  two      1  0.478943
4    a  one      4  1.965781)
('b',   key1 key2  data1     data2
2    b  one      2 -0.519439
3    b  two      3 -0.555730)


In [117]:
# Agregar (soma) a coluna 'data1' a partir do agrupamento da coluna 'key1'
soma_data1 = df['data1'].groupby(df['key1']).sum()
soma_data1

key1
a    5
b    5
Name: data1, dtype: int64

In [118]:
# Agrupamento com múltiplas colunas (key1 e key2)
# Se não passar nada para df, todas as colunas numericas são agregadas
media_data1 = df['data1'].groupby([df['key1'], df['key2']]).mean()
media_data1

key1  key2
a     one     2
      two     1
b     one     2
      two     3
Name: data1, dtype: int64

In [119]:
# Outra forma de fazer o mesmo comando acima
# as_index=False faz com que as colunas agrupadas seja tratadas como colunas e não como indices
df.groupby(['key1', 'key2'], as_index=False)[['data1']].mean()

,key1,key2,data1
0,a,one,2
1,a,two,1
2,b,one,2
3,b,two,3


In [120]:
# usando unstack para pivotear o indice mais interno para colunas e criar uma matriz
media_data1.unstack()

key2,one,two
key1,,
a,2,1
b,2,3


In [121]:
# tamanho dos grupos, quantidade de linhas que foram agrupadas em cada grupo

# df.groupby(["key1", "key2"])[["data1", "data2"]].size()

df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [122]:
# se não passar uma coluna numerica para df todas as colunas serão agregadas
# data1 e data2 são agregadas
df.groupby(df["key1"]).sum()

# df.groupby("key1")[["data1", "data2"]].sum()

,data1,data2
key1,,
a,5,2.240016
b,5,-1.075169


## Iterando sob grupos
Antes de aplicar a função de agregação os grupos são separados em varios subgrupos de acordo com a combinação de colunas que foram usadas no groupby. Esses gupos podem ser acessados antes de aplicar a agregação.
> o groupby devolve um objeto iteravel

In [123]:
# Iterando nos grupos e acessadi a tupla (o primeiro elemento é a coluna usada para agrupar e o segundo são os registros)
# Fazer a iteração antes de aplicar a função de agregação
for nome, grupo in df.groupby(['key1', 'key2']):
  print(f"\nColunas agrupadas: {nome}")
  print(grupo)
  


Colunas agrupadas: ('a', 'one')
  key1 key2  data1     data2
0    a  one      0 -0.204708
4    a  one      4  1.965781

Colunas agrupadas: ('a', 'two')
  key1 key2  data1     data2
1    a  two      1  0.478943

Colunas agrupadas: ('b', 'one')
  key1 key2  data1     data2
2    b  one      2 -0.519439

Colunas agrupadas: ('b', 'two')
  key1 key2  data1    data2
3    b  two      3 -0.55573


In [124]:
# criando um dicionário onde a chave é uma tupla com as colunas agrupadas e os valores são as linhas dos subgrupos
dicionario_grupos = dict(list(df.groupby(['key1', 'key2'])))

# acessando um subconjunto pela chave dos agrupamentos
dicionario_grupos[('a', 'one')]

,key1,key2,data1,data2
0,a,one,0,-0.204708
4,a,one,4,1.965781


In [125]:
# agrupando pelos tipos de colunas no eixo das colunas

# o df tem duas colunas float e duas object
print(f"Tipos de dados:\n{df.dtypes}\n")

# separar em dois grupos as colunas de float e as de object. axis=1 faz pelas colunas
grouped_columns = df.groupby(df.dtypes, axis=1)

# cada grupos apenas com colunas do mesmo tipo
for tipo, dados in grouped_columns:
  print(tipo)
  print(dados)

Tipos de dados:
key1      object
key2      object
data1      int64
data2    float64
dtype: object

int64
   data1
0      0
1      1
2      2
3      3
4      4
float64
      data2
0 -0.204708
1  0.478943
2 -0.519439
3 -0.555730
4  1.965781
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [126]:
# Tranforma as tuplas em uma lista e acessa somente as colunas object
list(grouped_columns)[2]

(dtype('O'),   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one)

## Agrupando com dicts mapeados
Passa um dicionário que determina a combinação de cada grupo, é como se estivessemos definindo um agrupamento sem que tenha um coluna no df, como um agrupamento externo usando um dict como referência

In [127]:
pessoas = pd.DataFrame(data=np.random.randn(5, 5),
                       columns=['a', 'b', 'c', 'd', 'e'],
                       index=["Leandro", "Bruna", "Breno", "João", "Maria"])
pessoas

,a,b,c,d,e
Leandro,1.393406,0.092908,0.281746,0.769023,1.246435
Bruna,1.007189,-1.296221,0.274992,0.228913,1.352917
Breno,0.886429,-2.001637,-0.371843,1.669025,-0.438570
João,-0.539741,0.476985,3.248944,-1.021228,-0.577087
Maria,0.124121,0.302614,0.523772,0.000940,1.343810


In [128]:
dict_mapeamento_grupos = {"a": "Red", "b": "Red", "c": "Red", "d": "Blue", "e": "Blue", "f":"Orange"}
dict_mapeamento_grupos

{'a': 'Red', 'b': 'Red', 'c': 'Red', 'd': 'Blue', 'e': 'Blue', 'f': 'Orange'}

In [129]:
# as colunas pode ser agrupadas de acordo com as combinações do dict
# cada chave do dict é mapeada com cada coluna e é retornado o valor o dict como agrupamento
agrupamento_por_colunas = pessoas.groupby(dict_mapeamento_grupos, axis=1)

# aplica uma agregação no agrupamento. Por exemplo, soma a, b, c na cor Red
print(f"Quantidade de colunas de cada cor:\n{agrupamento_por_colunas.size()}\n")

# imprime os valores dos grupos antes da agregação
print(f"\n{ [ print(grupo[1]) for grupo in agrupamento_por_colunas]}\n")

agrupamento_por_colunas.sum()

Quantidade de colunas de cada cor:
Blue    2
Red     3
dtype: int64

                d         e
Leandro  0.769023  1.246435
Bruna    0.228913  1.352917
Breno    1.669025 -0.438570
João    -1.021228 -0.577087
Maria    0.000940  1.343810
                a         b         c
Leandro  1.393406  0.092908  0.281746
Bruna    1.007189 -1.296221  0.274992
Breno    0.886429 -2.001637 -0.371843
João    -0.539741  0.476985  3.248944
Maria    0.124121  0.302614  0.523772

[None, None]



,Blue,Red
Leandro,2.015457,1.768060
Bruna,1.581830,-0.014040
Breno,1.230456,-1.487051
João,-1.598315,3.186187
Maria,1.344750,0.950507


In [130]:
# o mapeamento também funciona se for feito com um obj series

# tranforma o dict em series
map_series = pd.Series(dict_mapeamento_grupos)

# utiliza uma series para fazer o mapeamento
pessoas.groupby(map_series, axis=1).sum()

,Blue,Red
Leandro,2.015457,1.768060
Bruna,1.581830,-0.014040
Breno,1.230456,-1.487051
João,-1.598315,3.186187
Maria,1.344750,0.950507


## Agrupamentos com funções

In [131]:
# agrupa pelo tamanho do nome da pessoa (indice)
pessoas.groupby(len).sum()

,a,b,c,d,e
4,-0.539741,0.476985,3.248944,-1.021228,-0.577087
5,2.017740,-2.995245,0.426921,1.898878,2.258157
7,1.393406,0.092908,0.281746,0.769023,1.246435


In [132]:
# agrupando usando um nível de índice
indices_2niveis = pd.MultiIndex.from_arrays([["Leandro", "Bruna", "Breno", "João", "Maria"],
                                 ["natacao", "natacao", "natacao", "basquete", "atletismo"]], 
                                names=["familia", "esportes"])

# atribui o nome indice com 2 niveis para o dataframe
pessoas.index = indices_2niveis

# inverte os indices
pessoas.swaplevel("esportes", "familia")

a         b         c         d         e
esportes  familia                                                  
natacao   Leandro  1.393406  0.092908  0.281746  0.769023  1.246435
          Bruna    1.007189 -1.296221  0.274992  0.228913  1.352917
          Breno    0.886429 -2.001637 -0.371843  1.669025 -0.438570
basquete  João    -0.539741  0.476985  3.248944 -1.021228 -0.577087
atletismo Maria    0.124121  0.302614  0.523772  0.000940  1.343810

In [133]:
# agrupa pelo indice no level "esportes" apenas nas colunas numericas a e b
pessoas.groupby(level="esportes", axis=0)[["a", "b"]].sum()

,a,b
esportes,,
atletismo,0.124121,0.302614
basquete,-0.539741,0.476985
natacao,3.287025,-3.204951


# Agregação
Depois que os grupos são criados pela função groupby as funções de agregação entram em ação fazendo os calculos.
- count
- min, max
- str e var (desvio padrão e variancia)
- prod (produto dois valores)
- first, last

In [134]:
# soma
pessoas.reset_index()
pessoas.groupby(["esportes"])[["a", "b"]].sum()

,a,b
esportes,,
atletismo,0.124121,0.302614
basquete,-0.539741,0.476985
natacao,3.287025,-3.204951


In [135]:
# min
pessoas.groupby(["esportes"])[["a", "b"]].min()

,a,b
esportes,,
atletismo,0.124121,0.302614
basquete,-0.539741,0.476985
natacao,0.886429,-2.001637


In [136]:
# agg: usando suas proprias funções, ou uma lista de funções
# para cada coluna agregada, são aplicadas varias funções de agregação diferentes
pessoas.groupby("esportes").agg(["min", "max", "count", "sum"])[["a", "b"]]

a                  ...         b                
                min       max count  ...       max count       sum
esportes                             ...                          
atletismo  0.124121  0.124121     1  ...  0.302614     1  0.302614
basquete  -0.539741 -0.539741     1  ...  0.476985     1  0.476985
natacao    0.886429  1.393406     3  ...  0.092908     3 -3.204951

[3 rows x 8 columns]

In [137]:
def max_min_dif(dados):
  return dados.max() - dados.min()

# Agrupa os esportes e faz a agregação das colunas "a" e "b" aplicando as funções de agregação "max", "min", "max_min_dif" (customizada)
pessoas.groupby("esportes").agg(["count", "max", "min", max_min_dif])[["a", "b"]]

a                      ...         b                      
          count       max       min  ...       max       min max_min_dif
esportes                             ...                                
atletismo     1  0.124121  0.124121  ...  0.302614  0.302614    0.000000
basquete      1 -0.539741 -0.539741  ...  0.476985  0.476985    0.000000
natacao       3  1.393406  0.886429  ...  0.092908 -2.001637    2.094545

[3 rows x 8 columns]

In [138]:
pessoas.groupby("esportes").agg(["count", "mean", "std", "min", "quantile", "max" ])[["a"]]

a                                                  
          count      mean       std       min  quantile       max
esportes                                                         
atletismo     1  0.124121       NaN  0.124121  0.124121  0.124121
basquete      1 -0.539741       NaN -0.539741 -0.539741 -0.539741
natacao       3  1.095675  0.264818  0.886429  1.007189  1.393406

In [139]:
# definindo o nome da função
# por padrão os nome das colunas recebem os nome das funções utilizadas
# para alterar os nomes passamos as tuplas na função agg
pessoas.groupby("esportes").agg([("Máximo", "max"), ("Mínimo", "min"), ("Quantidade", "count"), ("Mediana", "quantile")])["a"]

,Máximo,Mínimo,Quantidade,Mediana
esportes,,,,
atletismo,0.124121,0.124121,1,0.124121
basquete,-0.539741,-0.539741,1,-0.539741
natacao,1.393406,0.886429,3,1.007189


In [140]:
# todas as 3 funções de agregação são aplicadas as colunas a e b
func = ("count", "max", "min")
resultado = pessoas[["a", "b"]].groupby("esportes").agg(func)

resultado["a"]

,count,max,min
esportes,,,
atletismo,1,0.124121,0.124121
basquete,1,-0.539741,-0.539741
natacao,3,1.393406,0.886429


In [142]:
# Para cada coluna numérica uma ou varias funções de agregação diferentes
pessoas.reset_index(inplace=True)

# agrupa os esportes e agrega para a coluna "a" usando a função sum e a coluna "b" usando as funções count, max, min
pessoas.groupby("esportes").agg({"a": "sum", "b": ["count", "max", "min"]})

a     b                    
                sum count       max       min
esportes                                     
atletismo  0.124121     1  0.302614  0.302614
basquete  -0.539741     1  0.476985  0.476985
natacao    3.287025     3  0.092908 -2.001637